In [18]:
# 필요한 라이브러리
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [11]:
# 캘리포니아 주택 가격 데이터: 20640 샘플, 8 특성
housing = fetch_california_housing()
housing.data.shape

(20640, 8)

In [12]:
# 특성 확인
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [13]:
# 훈련, 검증, 테스트 분리
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test  = scaler.transform(X_test)

X_train.shape, X_valid.shape, X_test.shape

((11610, 8), (3870, 8), (5160, 8))

In [14]:
# 심층신경망 구축
input_A = keras.layers.Input(shape = [5], name = "wide_input")
input_B = keras.layers.Input(shape = [6], name = "deep_input")
hidden1 = keras.layers.Dense(30, activation = "relu")(input_B)
hidden2 = keras.layers.Dense(30, activation = "relu")(hidden1)
concat  = keras.layers.concatenate([input_A, hidden2])
output  = keras.layers.Dense(1, name = "output")(concat)
model   = keras.Model(inputs = [input_A, input_B], outputs = [output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           930         dense_2[0][0]                    
____________________________________________________________________________________________

In [15]:
# 모델 컴파일 및 입력 데이터 생성
model.compile(loss = "mse", optimizer = keras.optimizers.SGD(lr = 1e-3))

X_train_A, X_train_B = X_train[:, : 5], X_train[:, -6 :]
X_valid_A, X_valid_B = X_valid[:, : 5], X_valid[:, -6 :]
X_test_A , X_test_B  = X_test[:, : 5] , X_test[:, -6 :]
X_train_A.shape, X_train_B.shape

((11610, 5), (11610, 6))

In [16]:
# 텐서보드에 필요한 로그폴더 생성
def get_run_logdir():
    import os
    import time
    root_logdir = os.path.join(os.curdir, "my_logs")
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [17]:
# 콜백 생성 및 모델 훈련, 검증
checkpoint_cb     = keras.callbacks.ModelCheckpoint("./checkpoint/my_keras_model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
tensorboard_cb    = keras.callbacks.TensorBoard(get_run_logdir())

history  = model.fit((X_train_A, X_train_B), y_train, validation_data = ((X_valid_A, X_valid_B), y_valid),\
                     epochs = 100, verbose = 1, callbacks = [checkpoint_cb, early_stopping_cb, tensorboard_cb])

mse_test = model.evaluate((X_test_A, X_test_B), y_test)

Epoch 1/100
363/363 [==============================] - 1s 3ms/step - loss: 4.1076 - val_loss: 1.0410
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.9768 - val_loss: 0.7043
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.7168 - val_loss: 0.5980
Epoch 4/100
363/363 [==============================] - 0s 999us/step - loss: 0.6284 - val_loss: 0.5470
Epoch 5/100
363/363 [==============================] - 0s 980us/step - loss: 0.5754 - val_loss: 0.5156
Epoch 6/100
363/363 [==============================] - 0s 999us/step - loss: 0.5514 - val_loss: 0.4950
Epoch 7/100
363/363 [==============================] - 0s 997us/step - loss: 0.5308 - val_loss: 0.4812
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5104 - val_loss: 0.4719
Epoch 9/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4840 - val_loss: 0.4630
Epoch 10/100
363/363 [==============================] - 0s 980us/step - loss: 0.511

In [ ]:
# 텐서보드: tensorboard --logdir=./my_logs --port=6006